# Analytics

#### Date: 2021/02

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM: Povos e Comunidades Tradicionais

##### Semester: 2021/01
##### Professor: Hilmer Neri

##### Members:

- Caio Vinicius F. de Araujo
- Guilherme Mendes Pereira
- Julio Cesar Litwin Lima
- Lucas Gomes de Oliveira
- Lucas Gomes Silva
- Lucas Maciel Aguiar
- Miguel Henrique de O. Alves

### LIBRARIES

In [ ]:
# Deal with data
import pandas as pd
import numpy as np
import json
from glob import glob
import os

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Deal with time
import datetime

### GRAPH SETTINGS

In [ ]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

### DATAFRAME SETTINGS

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

### SonarCloud

##### Path to the folder with all your jsons

In [ ]:
jsons = glob('../analytics-raw-data/*.json') # add the path here

In [ ]:
def read_json(json_path):
    
    with open(json_path) as json_file:
        json_obj = json.load(json_file)
        
    return json_obj

def create_base_component_df(json_list):
    
    df = pd.DataFrame()

    for i in json_list:

        base_component = read_json(i)

        base_component_data = base_component['baseComponent']['measures']

        base_component_df = pd.DataFrame(base_component_data)

        base_component_df['filename'] = os.path.basename(i)

        df = df.append(base_component_df, ignore_index=True)
        
    aux_df = df['filename'].str.split(r"PCTs-(.*?)-*-(.*?).json", expand=True)
    
    df['repository'] = aux_df[1]
    
    df['version'] = aux_df[2]
    
    df = df.sort_values(by=['repository', 'version'])
        
    return df

#### Create base component dataframe

In [ ]:
base_component_df = create_base_component_df(jsons)

In [ ]:
base_component_df.head(10)

#### Create dataframe per file

In [ ]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'coverage',
               'ncloc',
               'security_rating',
               'tests',
               'test_success_density',
               'test_execution_time',
               'reliability_rating']

len(metric_list)

In [ ]:
def metric_per_file(json):
    
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

def generate_file_dataframe_per_release(metric_list, json, language_extension):
    
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

def create_file_df(json_list):
    
    df = pd.DataFrame()

    for i in json_list:

        file_component = read_json(i)
        
        file_component_data = metric_per_file(file_component)

        file_component_df_ts = generate_file_dataframe_per_release(metric_list, file_component_data, language_extension = 'ts')
        file_component_df_ts['filename'] = os.path.basename(i)

        file_component_df_js = generate_file_dataframe_per_release(metric_list, file_component_data, language_extension = 'js')
        file_component_df_js['filename'] = os.path.basename(i)

        file_component_df_py = generate_file_dataframe_per_release(metric_list, file_component_data, language_extension = 'py')
        file_component_df_py['filename'] = os.path.basename(i)

        file_component_df_yaml = generate_file_dataframe_per_release(metric_list, file_component_data, language_extension = 'yaml')
        file_component_df_yaml['filename'] = os.path.basename(i)

        df = df.append(file_component_df_ts, ignore_index=True)
        df = df.append(file_component_df_js, ignore_index=True)
        df = df.append(file_component_df_py, ignore_index=True)
        df = df.append(file_component_df_yaml, ignore_index=True)
        
    aux_df = df['filename'].str.split(r"PCTs-(.*?)-(.*?).json", expand=True)
    
    df['repository'] = aux_df[1]
    
    df['version'] = aux_df[2]
    
    df = df.sort_values(by=['repository', 'version'])
        
    return df

In [ ]:
file_component_df = create_file_df(jsons)

In [ ]:
file_component_df.head(10)

#### Create dataframe per repository

In [ ]:
scraper_df = file_component_df[file_component_df['repository'] == 'Scraper']
documentsapi_df = file_component_df[file_component_df['repository'] == 'Documents-API']
mltraining_df = file_component_df[file_component_df['repository'] == 'ML-Training']
frontend_df = file_component_df[file_component_df['repository'] == 'Frontend']
usersapi_df = file_component_df[file_component_df['repository'] == 'Users-API']

In [ ]:
def _ncloc(df):
    ncloc = 0
    for each in df['ncloc']:
        n = 0
        try:
            n = int(each)
        except ValueError:
            n = 0
        ncloc += int(n)

    return ncloc

### Measure calculations according Q-Rapids quality model

# Quality Aspect - Maintainability
## Factor - Code Quality

##### COMPLEXITY

In [ ]:
def m1(df):
    
    density_non_complex_files = len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df)
    
    return density_non_complex_files

##### COMMENTS

In [ ]:
def m2(df):
    
    density_comment_files = len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df)
    
    return density_comment_files

##### DUPLICATIONS

In [ ]:
def m3(df):
    
    duplication = len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df)
    
    return duplication

# Quality Aspect - Reliability
## Factor - Testing Status

##### Passed tests

In [ ]:
def m4(df):

    passed_tests = df['test_success_density'].astype(float).median() / 100

    return passed_tests

##### Fast test builds

In [ ]:
def m5(df):

    density_fast_test_builds = len(df[(df['test_execution_time'].astype(float)) < 300]) / sum(df['tests'].astype(float))

    return density_fast_test_builds

##### Test coverage

In [ ]:
def m6(df):

    density_test_coverage = len(df[(df['coverage'].astype(float) > 60)]) / len(df)

    return density_test_coverage

### Calculate m1, m2, m3, m4, m5 and m6 for each repository

In [ ]:
def create_metrics_df(df):
    
    version_vec = df['version'].unique()
    
    m1_list = []
    m2_list = []
    m3_list = []
    m4_list = []
    m5_list = []
    m6_list = []

    ncloc_list = []
    repository_list = []
    version_list = []
    
    metrics_df = pd.DataFrame()
    
    for version in version_vec:

        version_df = df[df['version'] == version]

        m1_list.append(m1(version_df))
        m2_list.append(m2(version_df))
        m3_list.append(m3(version_df))
        m4_list.append(m3(version_df))
        m5_list.append(m3(version_df))
        m6_list.append(m3(version_df))

        ncloc_list.append(_ncloc(version_df))
        repository_list.append(version_df['repository'].iloc[0])
        version_list.append(version)
        
    metrics_df = pd.DataFrame({'m1': m1_list,
                               'm2': m2_list,
                               'm3': m3_list,
                               'm4': m3_list,
                               'm5': m3_list,
                               'm6': m3_list,
                               'repository': repository_list, 
                               'version': version_list,
                               'ncloc': ncloc_list})
        
    return metrics_df

In [ ]:
scraper_metrics = create_metrics_df(scraper_df)
documentsapi_metrics = create_metrics_df(documentsapi_df)
mltraining_metrics = create_metrics_df(mltraining_df)
frontend_metrics = create_metrics_df(frontend_df)
usersapi_metrics = create_metrics_df(usersapi_df)

### Data visualization

- You must do this for each of your repositories

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot(scraper_metrics['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(scraper_metrics['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(scraper_metrics['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(scraper_metrics['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(scraper_metrics['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(scraper_metrics['m6'], linewidth=3, marker='o', markersize=10)

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot(documentsapi_metrics['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(documentsapi_metrics['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(documentsapi_metrics['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(documentsapi_metrics['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(documentsapi_metrics['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(documentsapi_metrics['m6'], linewidth=3, marker='o', markersize=10)

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot(mltraining_metrics['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(mltraining_metrics['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(mltraining_metrics['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(mltraining_metrics['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(mltraining_metrics['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(mltraining_metrics['m6'], linewidth=3, marker='o', markersize=10)

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot(frontend_metrics['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(frontend_metrics['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(frontend_metrics['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(frontend_metrics['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(frontend_metrics['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(frontend_metrics['m6'], linewidth=3, marker='o', markersize=10)

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot(usersapi_metrics['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(usersapi_metrics['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(usersapi_metrics['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(usersapi_metrics['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(usersapi_metrics['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(usersapi_metrics['m6'], linewidth=3, marker='o', markersize=10)



### Sub characteristic aggregation

- You must do this for each of your repositories

In [ ]:
psc1 = 1
psc2 = 1
pc1 = 0.5
pc2 = 0.5
pm1 = 0.33
pm2 = 0.33
pm3 = 0.33
pm4 = 0.15
pm5 = 0.15
pm6 = 0.7

scraper_metrics['code_quality'] = ((scraper_metrics['m1']*pm1) + (scraper_metrics['m2']*pm2) + (scraper_metrics['m3']*pm3)) * psc1
documentsapi_metrics['code_quality'] = ((documentsapi_metrics['m1']*pm1) + (documentsapi_metrics['m2']*pm2) + (documentsapi_metrics['m3']*pm3)) * psc1
mltraining_metrics['code_quality'] = ((mltraining_metrics['m1']*pm1) + (mltraining_metrics['m2']*pm2) + (mltraining_metrics['m3']*pm3)) * psc1
frontend_metrics['code_quality'] = ((frontend_metrics['m1']*pm1) + (frontend_metrics['m2']*pm2) + (frontend_metrics['m3']*pm3)) * psc1
usersapi_metrics['code_quality'] = ((usersapi_metrics['m1']*pm1) + (usersapi_metrics['m2']*pm2) + (usersapi_metrics['m3']*pm3)) * psc1

scraper_metrics['testing_status'] = ((scraper_metrics['m4']*pm4) + (scraper_metrics['m5']*pm5) + (scraper_metrics['m6']*pm6)) * psc2
documentsapi_metrics['testing_status'] = ((documentsapi_metrics['m4']*pm4) + (documentsapi_metrics['m5']*pm5) + (documentsapi_metrics['m6']*pm6)) * psc2
mltraining_metrics['testing_status'] = ((mltraining_metrics['m4']*pm4) + (mltraining_metrics['m5']*pm5) + (mltraining_metrics['m6']*pm6)) * psc2
frontend_metrics['testing_status'] = ((frontend_metrics['m4']*pm4) + (frontend_metrics['m5']*pm5) + (frontend_metrics['m6']*pm6)) * psc2
usersapi_metrics['testing_status'] = ((usersapi_metrics['m4']*pm4) + (usersapi_metrics['m5']*pm5) + (usersapi_metrics['m6']*pm6)) * psc2


In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(scraper_metrics['code_quality'], linewidth=3, marker='o', markersize=5)
plt.plot(documentsapi_metrics['code_quality'], linewidth=3, marker='o', markersize=5)
plt.plot(mltraining_metrics['code_quality'], linewidth=3, marker='o', markersize=5)
plt.plot(frontend_metrics['code_quality'], linewidth=3, marker='o', markersize=5)
plt.plot(usersapi_metrics['code_quality'], linewidth=3, marker='o', markersize=5)


In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(scraper_metrics['testing_status'], linewidth=3, marker='o', markersize=5)
plt.plot(documentsapi_metrics['testing_status'], linewidth=3, marker='o', markersize=5)
plt.plot(mltraining_metrics['testing_status'], linewidth=3, marker='o', markersize=5)
plt.plot(frontend_metrics['testing_status'], linewidth=3, marker='o', markersize=5)
plt.plot(usersapi_metrics['testing_status'], linewidth=3, marker='o', markersize=5)


In [ ]:
metrics_df = pd.concat([scraper_metrics, documentsapi_metrics, mltraining_metrics, frontend_metrics, usersapi_metrics], ignore_index=True)

metrics_df['maintainability'] = metrics_df['code_quality'] * pc1
metrics_df['Reliability'] = metrics_df['testing_status'] * pc2
metrics_df['total'] = metrics_df['maintainability'] + metrics_df['Reliability']

In [ ]:
metrics_df

In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(metrics_df['maintainability'], linewidth=3, marker='o', markersize=5)
plt.plot(metrics_df['Reliability'], linewidth=3, marker='o', markersize=5)


plt.ylim(.45,.7)

In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(metrics_df['total'], linewidth=3, marker='o', markersize=5)


plt.ylim(.45,.7)


In [ ]:
# ANALYSIS_DATE FORMAT: MM-DD-YYYY
currentTime = datetime.datetime.now().strftime("%d-%m-%Y")

metrics_df.to_excel('./data/fga-eps-mds-2021_1-PCTs-DATASET-{}.xlsx'.format(currentTime), index = False)
metrics_df.to_csv('./data/fga-eps-mds-2021_1-PCTs-DATASET-{}.csv'.format(currentTime), index = False)